This notebook will allow us to input expenses in a group and then split it accordingly.

In [1]:
import pandas as pd
import numpy as np

In [2]:
class expense_group():
    '''
    Class to create a database of transactions.
    Additionally, computes the balances and simplifies debt.

    Attributes:
    -----------
    group_members: list of str
        names of people who are involved.
    expense_data: pandas.core.frame.DataFrame
        stores details about individual transactions.
    lender_list: list of tuple
        each item consists of the name of the lender and amount to be credited.
    borrower_list: list of tuple
        each item consists of the name of the borrower and amount to be debited.
    simplified_transactions: list of tuple
        each item is of the format (borrower, lender, amount).

    Methods:
    --------
    add_members(): 
        adds data to group_members
    add_expense():
        adds data row to expense_data
    delete_expense():
        deletes data row in expense_data
    simplify_debt():
        reduces the number of overall transactions
    '''
    def __init__(self):
        '''
        Constructor for the expense_group class
        '''
        self.group_members = []
        self.lender_list = None
        self.borrower_list = None
        self.simplified_transactions = None
        self.expense_data = pd.DataFrame(columns=["Expenditure name", "Paid by", "Total", "Debit-Credit"])
        self.add_members()

    def _update_expenses(self):
        ''' Update the expense_data when new members are added. '''
        for i in range(len(self.expense_data)):
            existing_arr = self.expense_data.at[i, "Debit-Credit"]
            new_arr = np.append(existing_arr, [0])
            self.expense_data.at[i, "Debit-Credit"] = new_arr
        return
    
    def add_members(self):
        ''' Add data to group members. Requires user input. '''
        add_more = True
        print("\n---------- ADD MEMBER NAMES ----------\n")
        while add_more:
            print("Enter name of the member:")
            name = input()
            name = name.split()
            for n in name:
                n = n.capitalize()
                self.group_members.append(n)
                self._update_expenses()
                print(f"Hello {n}, you are added to the group.\n") 
            print("Would you like to add more members? (y/n)")
            choice = input().lower()
            if choice != 'y':
                add_more = False
        print("\nCurrent members are:", self.group_members)
        return
    
    def _update_lender_borrower_lists(self):
        ''' Updates the lender list and borrower list after every transaction. '''
        net = np.sum(self.expense_data["Debit-Credit"])
        self.borrower_list = []
        self.lender_list = []
        try:
            for i in zip(self.group_members, net):
                if np.sign(i[1])==-1:
                    self.borrower_list.append(i)
                elif np.sign(i[1])==1:
                    self.lender_list.append(i)
        except TypeError:
            print("Empty lender/borrower lists.")
        return
    
    def add_expense(self):
        ''' Adds data row (transaction) to expense data. Requires user input. '''
        data = {}
        print("\n---------- ADD EXPENDITURE DETAILS ----------\n")
        print("Enter name of the expenditure: ")
        exp_name = input().capitalize()
        print("What is the total expenditure: ")
        try: 
            exp_amt = round(float(input()),2)
        except ValueError:
            print("Invalid Response. ABORTING...")
            return
        print("Paid by: ")
        lender = input().capitalize()
        if lender in self.group_members:
            lender_idx = self.group_members.index(lender)
            dr_cr = self._divide_expense(lender_idx, exp_amt)
            if np.any(dr_cr):
                self.expense_data.loc[len(self.expense_data.index)] = [exp_name, lender, exp_amt, dr_cr] 
        else:
            print("Member doesn't exist. ABORTING...")
            return
        self._update_lender_borrower_lists()       
        return
        
    def _divide_equally(self, member_idx, amount):
        ''' Returns array of balances for a transaction divided equally among members. '''
        amount_per_person = round(amount/(len(self.group_members)),2)
        dr_cr = np.zeros(len(self.group_members))
        dr_cr[member_idx] = amount
        dr_cr -= amount_per_person
        return dr_cr
    
    def _divide_equally_subgroups(self, member_idx, amount):
        ''' 
        Returns array of balances for a transaction divided 
        equally among subgroup of members. Requires user input.
        '''
        dr_cr = np.zeros(len(self.group_members))
        print("\tEnter the names of creditors separated by spaces:")
        borrowers = input().split()
        amount_per_person = round(amount/(len(borrowers)),2)
        if (len(borrowers)==1) and (borrowers[0].capitalize()==self.group_members[member_idx]):
            print("\tInvalid response. ABORTING ...")
            return dr_cr
        for borrower in borrowers:
            borrower = borrower.capitalize()
            if borrower in self.group_members:
                borrower_idx = self.group_members.index(borrower)
                dr_cr[borrower_idx] -= amount_per_person
            else:
                print("\n\tMember not found. ABORTING...")
                return np.zeros(len(self.group_members))
        dr_cr[member_idx] += amount
        return dr_cr

    def _divide_unequally(self, member_idx, amount):
        '''
        Returns array of balances for a transaction divided unequally among members.
        Requires user input.
        '''
        dr_cr = np.zeros(len(self.group_members))
        dr_cr[member_idx] = amount
        remaining = amount
        for borrower in self.group_members[:-1]:
            print(f"\tOf Rs.{remaining}, {borrower} owes:")
            debit = round(float(input()),2)
            while debit > remaining:
                print("\tInvalid input. Debit should be less than the remaining credit.")
                print(f"\tOf Rs.{remaining}, {borrower} owes:")
                debit = round(float(input()),2)
            remaining -= debit
            borrower_idx = self.group_members.index(borrower)
            dr_cr[borrower_idx] -= debit
        print(f"\t{self.group_members[-1]} owes Rs{remaining}")
        dr_cr[-1] -= remaining
        return dr_cr
    
    def _divide_by_percentages(self, member_idx, amount):
        '''
        Returns array of balances for a transaction divided by percentages among members.
        Requires user input.
        '''
        dr_cr = np.zeros(len(self.group_members))
        dr_cr[member_idx] = amount
        remaining_pc = 100
        for borrower in self.group_members[:-1]:
            print(f"\tOf {remaining_pc}%, {borrower} owes (in percentages):")
            debit_pc = round(float(input()),2)
            while debit_pc > remaining_pc:
                print("\tInvalid input. Debit cannot be greater than the remaining credit.")
                print(f"\tOf {remaining_pc}%, {borrower} owes (in percentages):")
                debit_pc = round(float(input()),2)
            debit = debit_pc * amount / 100
            remaining_pc -= debit_pc
            borrower_idx = self.group_members.index(borrower)
            dr_cr[borrower_idx] -= round(debit,2)
        print(f"\t{self.group_members[-1]} owes {round(remaining_pc, 2)}% of {amount}")
        dr_cr[-1] -= round(remaining_pc/100 * amount, 2)
        return dr_cr

    def _divide_expense(self, member_idx, amount):
        ''' Returns array of balances for a transaction. Requires user input. '''
        print("\n\tHow would you like to divide the expenditure?\n")
        print("\tA - Equally among the members\n")
        print("\tB - Equally among a subset of members\n")
        print("\tC - Unequally among the members\n")
        print("\tD - By percentages\n")
        print("\tEnter your choice: ")
        choice = input().lower()
        if choice == 'a':
            dr_cr = self._divide_equally(member_idx, amount)
        elif choice == 'b':
            dr_cr = self._divide_equally_subgroups(member_idx, amount)
        elif choice == 'c':
            dr_cr = self._divide_unequally(member_idx, amount)
        elif choice == 'd':
            dr_cr = self._divide_by_percentages(member_idx, amount)
        return dr_cr
    
    def delete_expense(self):
        ''' Deletes data row (transaction) in expense data. Requires user input. '''
        print("\n\tEnter the name of the expense that you'd like to delete: ")
        exp_name = input().capitalize()
        if exp_name in self.expense_data["Expenditure name"].values:
            self.expense_data.drop(self.expense_data[
                self.expense_data["Expenditure name"]==exp_name].index, inplace=True)
            print(f"\t{exp_name} deleted.")
            self._update_lender_borrower_lists()
        else:
            print(f"\t{exp_name} not found. ABORTING ...")
        return
    
    def simplify_debt(self, verbose=True):
        ''' Reduces the number of transactions '''
        self.simplified_transactions = []
        if self.lender_list and self.borrower_list:
            positive_balances = {person: balance for person, balance in self.lender_list}
            negative_balances = {person: - balance for person, balance in self.borrower_list}
            while positive_balances and negative_balances:
                max_creditor = max(positive_balances, key=positive_balances.get)
                max_debtor = max(negative_balances, key=negative_balances.get)
                min_amount = min(positive_balances[max_creditor], negative_balances[max_debtor])
                self.simplified_transactions.append((max_debtor, max_creditor, round(min_amount,2)))
                positive_balances[max_creditor] -= min_amount
                negative_balances[max_debtor] -= min_amount
                if positive_balances[max_creditor] == 0:
                    del positive_balances[max_creditor]
                if negative_balances[max_debtor] == 0:
                    del negative_balances[max_debtor]
        else:
            print("No transactions found. ABORTING ...")
        return

In [6]:
new_group = expense_group()


---------- ADD MEMBER NAMES ----------

Enter name of the member:
Hello Ankur, you are added to the group.

Hello Anup, you are added to the group.

Hello Bhanu, you are added to the group.

Hello Koustav, you are added to the group.

Hello Manish, you are added to the group.

Hello Swapan, you are added to the group.

Hello Uddeepta, you are added to the group.

Would you like to add more members? (y/n)

Current members are: ['Ankur', 'Anup', 'Bhanu', 'Koustav', 'Manish', 'Swapan', 'Uddeepta']


In [4]:
new_group.simplify_debt()

No transactions found. ABORTING ...


In [30]:
new_group.add_expense()


---------- ADD EXPENDITURE DETAILS ----------

Enter name of the expenditure: 
What is the total expenditure: 
Paid by: 

	How would you like to divide the expenditure?

	A - Equally among the members

	B - Equally among a subset of members

	C - Unequally among the members

	D - By percentages

	Enter your choice: 
	Of Rs.40.0, Ankur owes:
	Of Rs.40.0, Anup owes:
	Of Rs.20.0, Bhanu owes:
	Of Rs.20.0, Koustav owes:
	Of Rs.10.0, Manish owes:
	Of Rs.0.0, Swapan owes:
	Uddeepta owes Rs0.0


In [31]:
new_group.expense_data

,Expenditure name,Paid by,Total,Debit-Credit
0,Classic booking,Uddeepta,1984.0,"[0.0, 0.0, -992.0, 0.0, 0.0, 0.0, 992.0]"
1,Himalayan booking,Bhanu,2560.0,"[0.0, 0.0, 1280.0, 0.0, 0.0, 0.0, -1280.0]"
2,Classic fuel,Uddeepta,800.0,"[-400.0, 0.0, 0.0, 0.0, 0.0, -400.0, 800.0]"
3,Himalayan fuel,Bhanu,1200.0,"[-600.0, 0.0, 1200.0, 0.0, 0.0, -600.0, 0.0]"
4,Jupiter petrol,Ankur,300.0,"[300.0, 0.0, 0.0, -150.0, -150.0, 0.0, 0.0]"
5,Parking,Ankur,60.0,"[51.43, -8.57, -8.57, -8.57, -8.57, -8.57, -8.57]"
6,Breakfast,Ankur,80.0,"[40.0, 0.0, 0.0, 0.0, 0.0, 0.0, -40.0]"
7,Extra dosa,Ankur,30.0,"[30.0, 0.0, 0.0, 0.0, -30.0, 0.0, 0.0]"
8,Coconuts,Ankur,200.0,"[150.0, 0.0, 0.0, -50.0, -50.0, -50.0, 0.0]"
9,Jupiter rent,Koustav,1810.0,"[0.0, 0.0, 0.0, 905.0, -905.0, 0.0, 0.0]"


In [32]:
print(new_group.lender_list)
print(new_group.borrower_list)

[('Bhanu', 1479.43), ('Koustav', 806.4300000000001), ('Uddeepta', 463.43)]
[('Ankur', -468.57000000000005), ('Anup', -28.57), ('Manish', -1153.57), ('Swapan', -1098.5700000000002)]


In [35]:
new_group.simplify_debt()

In [36]:
new_group.simplified_transactions

[('Manish', 'Bhanu', 1153.57),
 ('Swapan', 'Koustav', 806.43),
 ('Ankur', 'Uddeepta', 463.43),
 ('Swapan', 'Bhanu', 292.14),
 ('Anup', 'Bhanu', 28.57),
 ('Ankur', 'Bhanu', 5.14)]

In [72]:
new_group.delete_expense()


	Enter the name of the expense that you'd like to delete: 
	Travel not found. ABORTING ...
